In [1]:
#1. 사용환경 준비
import os
from getpass import getpass
import requests

os.environ["OPENAI_API_KEY"] = getpass("OpenAI API key 입력: ") # API 키 입력


In [2]:
#2. 모델 로드하기 
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

# 모델 초기화
model = ChatOpenAI(model="gpt-4o-mini")

In [1]:
#3. 문서 로드하기
from langchain.document_loaders import PyPDFLoader

# PDF 파일 로드. 파일의 경로 입력
loader = PyPDFLoader("인공지능산업최신동향_2024년11월호.pdf")

# 페이지 별 문서 로드
docs = loader.load()
print(docs[:1])


[Document(metadata={'source': '인공지능산업최신동향_2024년11월호.pdf', 'page': 0}, page_content='2024년 11월호')]


In [8]:
#4. 문서 청크로 나누기(CharacterTextSplitter)
from langchain.text_splitter import CharacterTextSplitter

# 텍스트 청크 분할기 설정 (문단 기준 분할)
text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=100,
    chunk_overlap=10,
    length_function=len,
    is_separator_regex=False,
)

splits = text_splitter.split_documents(docs) # 문서를 청크로 분할
print(splits[:1]) # 상위 10개만 출력

[Document(metadata={'source': '인공지능산업최신동향_2024년11월호.pdf', 'page': 0}, page_content='2024년 11월호')]


# CharacterTextSplitter 청킹 방식 및 매개변수 설명

## 청킹 방식
`CharacterTextSplitter`는 텍스트를 지정된 기준에 따라 나누어 작은 조각(청크)으로 만드는 기능을 제공한다. 이 방식은 대량의 텍스트를 처리할 때 유용합니다.

## 매개변수 설명

1. **separator**: 
   - **설명**: 문서를 나눌 기준으로 사용할 문자열을 설정한다는 뜻이다.

2. **chunk_size**:
   - **설명**: 각 청크의 최대 크기를 설정한다. 청크는 이 크기를 초과하지 않는다.

3. **chunk_overlap**:
   - **설명**: 청크 간의 겹치는 글자 수를 설정한다. 문맥을 유지하는 데 도움이 된다.


4. **length_function**:
   - **설명**: 청크의 길이를 계산하는 함수이다. 일반적으로 문자열 길이를 계산하는 `len` 함수를 사용한다.


5. **is_separator_regex**:
   - **설명**: 구분자가 정규 표현식인지 여부를 설정한다. `False`로 설정하면 일반 문자열로 처리된다.


## 사용 목적
이 과정을 통해 대량의 텍스트를 효과적으로 처리하고, AI 모델이 이해할 수 있는 형태로 변환하여 정보 추출 및 분석을 용이하게 합니다.

In [9]:
#4. 문서 청크로 나누기(RecursiveCharacterTextSplitter)

from langchain.text_splitter import RecursiveCharacterTextSplitter

# # 텍스트 분할기 설정 (재귀적 문자 기반 분할)
recursive_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=10,
    length_function=len,
    is_separator_regex=False,
)

splits = recursive_text_splitter.split_documents(docs) # 문서를 청크로 분할
print(splits[:1]) # 상위 10개만 출력

[Document(metadata={'source': '인공지능산업최신동향_2024년11월호.pdf', 'page': 0}, page_content='2024년 11월호')]


# RecursiveCharacterTextSplitter 청킹 방식 및 매개변수 설명

## 청킹 방식
`RecursiveCharacterTextSplitter`는 텍스트를 재귀적으로 나누어 작은 조각(청크)으로 만드는 기능을 제공한다. 이 방식은 긴 문서나 복잡한 내용을 효과적으로 처리할 수 있다.

## 매개변수 설명

1. **chunk_size**:
   - **설명**: 각 청크의 최대 크기를 설정한다. 청크는 이 크기를 초과하지 않는다.

2. **chunk_overlap**:
   - **설명**: 청크 간의 겹치는 글자 수를 설정한다. 문맥을 유지하는 데 도움이 된다.

3. **length_function**:
   - **설명**: 청크의 길이를 계산하는 함수이다. 일반적으로 문자열 길이를 계산하는 `len` 함수를 사용한다.

4. **is_separator_regex**:
   - **설명**: 구분자가 정규 표현식인지 여부를 설정한다. `False`로 설정하면 일반 문자열로 처리된다.

## 사용 목적
이 과정을 통해 대량의 텍스트를 효과적으로 처리하고, AI 모델이 이해할 수 있는 형태로 변환하여 정보 추출 및 분석을 용이하게 한다. `RecursiveCharacterTextSplitter`는 문맥을 잘 유지하면서 청크를 나누기 때문에, 복잡한 텍스트에 특히 유용하다.


In [17]:
#5 벡터 임베딩 생성
from langchain_openai import OpenAIEmbeddings

# OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")


In [18]:
#6. 벡터 스토어 생성
import faiss
from langchain_community.vectorstores import FAISS

# 문서에서 벡터 저장소 생성
vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)


In [19]:
#7. FAISS를 Retriever로 변환
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 1})

In [20]:
#8. 프롬프트 템플릿을 정의하라
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# 프롬프트 템플릿 정의
contextual_prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the question using only the following context."),
    ("user", "Context: {context}\\n\\nQuestion: {question}")
])

In [21]:
#9. LangChain의 모델과 프롬프트를 연결하여 RAG 체인을 구성하라. 
from langchain.chains import LLMChain

class SimplePassThrough:
    def invoke(self, inputs, **kwargs):
        return inputs

class ContextToPrompt:
    def __init__(self, prompt_template):
        self.prompt_template = prompt_template
    
    def invoke(self, inputs):
        # 문서 내용을 텍스트로 변환
        if isinstance(inputs, list):
            context_text = "\n".join([doc.page_content for doc in inputs])
        else:
            context_text = inputs
        
        # 프롬프트 템플릿에 적용
        formatted_prompt = self.prompt_template.format_messages(
            context=context_text,
            question=inputs.get("question", "")
        )
        return formatted_prompt

# Retriever를 invoke() 메서드로 래핑하는 클래스 정의
class RetrieverWrapper:
    def __init__(self, retriever):
        self.retriever = retriever

    def invoke(self, inputs):
        if isinstance(inputs, dict):
            query = inputs.get("question", "")
        else:
            query = inputs
        # 검색 수행
        response_docs = self.retriever.get_relevant_documents(query)
        return response_docs
#텍스트 생성 체인 생성
llm_chain = LLMChain(llm=model, prompt=contextual_prompt)

# RAG 체인 설정
rag_chain_debug = {
    "context": RetrieverWrapper(retriever),
    "prompt": ContextToPrompt(contextual_prompt),
    "llm": model
}

## RAG(생성형 응답 생성)의 필요성

RAG는 정보 검색과 생성 모델을 결합하여 사용자 질문에 대한 보다 정확하고 관련성 높은 답변을 생성하는 기술이다. RAG의 필요성은 다음과 같은 이유로 설명할 수 있다.

1. **정보의 동적 접근**: RAG는 최신 정보를 실시간으로 검색할 수 있어, 사용자가 요청하는 질문에 대해 항상 최신의 정확한 정보를 제공할 수 있다.

2. **맥락 이해**: 사용자 질문에 대한 답변을 생성할 때, RAG는 관련된 맥락을 기반으로 하여 보다 깊이 있는 이해를 제공합니다. 이는 단순한 정답 제공을 넘어, 질문의 의도를 파악하여 적절한 답변을 생성하는 데 기여한다.

3. **대량의 데이터 처리**: RAG는 대량의 문서와 정보를 처리할 수 있는 능력을 가지고 있어, 사용자가 원하는 다양한 주제에 대해 신속하게 대응할 수 있다.

4. **개선된 사용자 경험**: RAG를 통해 제공되는 답변은 보다 개인화되고 관련성이 높아, 사용자 경험을 개선할 수 있다. 이는 사용자 만족도를 높이고, 반복적인 질문을 줄이는 데 도움이 된다.

결론적으로, RAG는 정보 검색과 생성 모델의 장점을 결합하여, 더 나은 품질의 질문 응답 시스템을 구축하는 데 필수적이다.

In [ ]:
#10. 질문에 응답하는 챗봇을 구동하여 질문해라. 

# 챗봇 구동
while True:
    print("========================")
    query = input("질문을 입력하세요 : ")
    if query == "종료": # 종료 입력 시 챗봇 종료
        break
    
    # 1. Retriever로 관련 문서 검색
    response_docs = rag_chain_debug["context"].invoke({"question": query})
    
    # 2. 문서를 프롬프트로 변환
    prompt_messages = rag_chain_debug["prompt"].invoke({
        "context": response_docs,
        "question": query
    })
    
    # 3. LLM으로 응답 생성
    response = rag_chain_debug["llm"].invoke(prompt_messages)
    
    print("\n답변:")
    print(response.content) # 답변 출력


답변:
인공지능은 컴퓨터 시스템이 인간의 지능을 모방하여 학습, 추론, 문제 해결 등의 작업을 수행할 수 있도록 하는 기술입니다.

답변:
문서는 공공 부문의 AI 도입을 안내하는 보고서로, AI 도입 시 단계별 접근방식을 권고합니다. 이 과정은 프로토타입부터 시작해 시범 도입을 거쳐 본격적으로 구현하는 방식으로 구성되어 있습니다.

답변:
저는 정보를 제공하고 질문에 답변하는 AI입니다.


In [23]:
#도전 과제 1. LangSmith의 Prompt Library 를 참고하여 프롬프트를 3개 이상 아래와 같은 파일 구조로 저장해라. 

import os
from langchain import hub

# 폴더 및 파일 이름 설정
folder_name = 'Prompts'
file_names = ['prompt1.txt', 'prompt2.txt', 'prompt3.txt']

# Prompts 폴더 생성
os.makedirs(folder_name, exist_ok=True)

# 외부 저장소에서 프롬프트를 가져옴
prompt1 = hub.pull("jisujiji/rag-prompt-1") # 첫 번째 프롬프트
prompt2 = hub.pull("jakab/rag-prompt") # 두 번째 프롬프트
prompt3 = hub.pull("rlm/rag-answer-hallucination") # 세 번째 프롬프트


# 프롬프트 파일 내용 작성
prompts = [
    str(prompt1), # 첫 번째 파일 내용
    str(prompt2), # 두 번째 파일 내용
    str(prompt3) # 세 번째 파일 내용
]

# 프롬프트 파일 생성
for file_name, prompt in zip(file_names, prompts):
    with open(os.path.join(folder_name, file_name), 'w', encoding='utf-8') as f:
        f.write(prompt)

print("폴더 및 파일이 성공적으로 생성되었습니다.") # 결과 출력

c:\Users\h\Desktop\TIL\.venv\Lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
c:\Users\h\Desktop\TIL\.venv\Lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
c:\Users\h\Desktop\TIL\.venv\Lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


폴더 및 파일이 성공적으로 생성되었습니다.


In [ ]:
#도전 과제 2. 각 프롬프트를 외부에서 불러와서 실행할 수 있도록 코드를 고쳐라. 
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
import os
import time

# 파일로 저장한 프롬프트를 가져옴
while True:
    try:
        a = input("가져올 프롬프트 이름: ") 
        file_path = f"C:\\Users\\h\\Desktop\\TIL\\personal_assignment\\Prompts\\{a}" # 파일 경로
        with open(file_path, "r") as f:
            prompt = f.read() # 파일을 읽어서 저장
        break
    except FileNotFoundError and OSError:
        print("없는 파일입니다!")

# 프롬프트 템플릿 생성
prompt_template = ChatPromptTemplate.from_template(prompt)
#텍스트 생성 체인 설정
llm_chain = LLMChain(llm=model, prompt=prompt_template)


#챗봇 구동
while True:
    
    query = input("질문을 입력하세요 : ")
    if query == "종료": # 종료 입력 시 챗봇 종료
        break
    
    # 1. Retriever로 관련 문서 검색
    response_docs = rag_chain_debug["context"].invoke({"question": query})
    
    # 2. 문서를 프롬프트로 변환
    prompt_messages = rag_chain_debug["prompt"].invoke({
        "context": response_docs,
        "question": query
    })
    
    # 3. LLM으로 응답 생성
    response = rag_chain_debug["llm"].invoke(prompt_messages)
    
    print("\n답변:")
    print(response.content) # 답변 출력

    # 도전 과제 3번 실행 결과는 자동으로 Result 디렉토리에 저장되어야 한다. 이때, 실험 결과 파일 이름은 실험에 쓰인 프롬프트의 이름과 timestamp을 포함해야한다.
    
    # 현재 타임스탬프 기록
    timestamp = time.time()

    # 폴더 및 파일 이름 설정
    folder_name = 'Results'
    file_name = f"{a[:7]}_result_{timestamp}.txt"

    # 폴더 생성
    os.makedirs(folder_name, exist_ok=True)

    # 답변 파일 생성
    with open(os.path.join(folder_name, file_name), 'w', encoding='utf-8') as f:
        f.write(response.content)



답변:
인공지능은 인간의 지능을 모방하여 학습, 문제 해결, 의사 결정 등의 작업을 수행할 수 있는 시스템이나 프로그램을 의미합니다.

답변:
문서 내용은 카카오가 카나나라는 앱을 카카오톡과 별개로 출시할 계획에 대한 정보와 함께, 이용자를 지원하기 위해 스터디 그룹 대화에서 함께 읽은 논문 관련 퀴즈를 제공하고 채점 및 부연 설명을 하는 기능에 대한 설명이 포함되어 있습니다.

답변:
'인공지능산업최신동향_2024년11월호.pdf'라는 문서를 참고했습니다.

답변:
저는 로제타폴드의 핵심개발자이자 제1저자인 백민경 교수와 허사비스입니다.
